In [18]:
import numpy as np
import datetime

In [19]:
x_train = np.load('data/train/x_train.npy')
y_train = np.load('data/train/y_train_seg.npy')

In [20]:
from keras import backend as K
from model_unet3d import unet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU

ir = x_train.shape[1] # rows
ic = x_train.shape[2] # cols
ih = x_train.shape[3] # slices

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, ir, ic, ih)
    input_shape = (1, ir, ic, ih)
else:
    x_train = x_train.reshape(x_train.shape[0], ir, ic, ih, 1)
    input_shape = (ir, ic, ih, 1)

model = unet(input_shape=input_shape)
model.summary()

model.compile(optimizer=Adam(), 
              loss=BinaryCrossentropy(), 
              metrics=[MeanIoU(num_classes=2)])

trainstarttime = datetime.datetime.today()
print("")
print("Train start time: " + str(trainstarttime))

history = model.fit(x_train, y_train, 
                    epochs=70, 
                    batch_size=1, 
                    verbose=1)

trainfinishtime = datetime.datetime.today()
print("Train finish time: " + str(trainfinishtime))
ttime = str(trainfinishtime - trainstarttime)
print("Training time: " + ttime)

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 256, 256, 1)]   0         []                            
                                                                                                  
 conv3d_2 (Conv3D)           (None, 256, 256, 256, 64)    1792      ['input_3[0][0]']             
                                                                                                  
 conv3d_3 (Conv3D)           (None, 256, 256, 256, 64)    110656    ['conv3d_2[0][0]']            
                                                                                                  
 max_pooling3d (MaxPooling3  (None, 128, 128, 128, 64)    0         ['conv3d_3[0][0]']            
 D)                                                                                            


Train start time: 2024-03-02 23:07:52.994858
Epoch 1/70


In [ ]:
file = 'data/train/models/unet_model'

json_string = model.to_json()
fj = file + '.json'
fh = file + '.h5'
open(fj, 'w').write(json_string)
model.save_weights(fh)

print("Model saved.")